In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torchvision.datasets import MNIST
import torchvision.transforms as transforms
from torch.utils.data import random_split, DataLoader
from torchvision.datasets.utils import download_url

import matplotlib.pyplot as plt
import tarfile

## Exploring MNIST dataset

### Downloading/Loading torchvision.dataset.MNIST as Tensors.

In [4]:
# Dowload the dataset
dataset_url = "https://s3.amazonaws.com/fast-ai-imageclas/cifar10.tgz"
download_url(dataset_url, '.')

In [6]:
# Extract from archive
with tarfile.open('./cifar10.tgz', 'r:gz') as tar:
    tar.extractall(path='./data')

In [12]:
# Viewing the data
data_dir = './data/cifar10'
print("Directories: ", os.listdir(data_dir))
classes = os.listdir(data_dir + "/train")
print("Classes: ", classes)

Directories:  ['test', 'train']
Classes:  ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


Checking some of the contents of the folders

In [13]:
airplane_files = os.listdir(data_dir + "/train/airplane")
print('No. of training examples for airplanes:', len(airplane_files))
print(airplane_files[:5])

No. of training examples for airplanes: 5000
['0001.png', '0002.png', '0003.png', '0004.png', '0005.png']
